$$\newcommand{\ket}[1]{\left|{#1}\right\rangle}$$
$$\newcommand{\bra}[1]{\left\langle{#1}\right|}$$
$$\newcommand{\braket}[2]{\left\langle{#1}\middle|{#2}\right\rangle}$$
$$\def\lc{\left\lceil}$$
$$\def\rc{\right\rceil}$$
# The Ising model

The Lenz-Ising model consists of discrete variables that represent magnetic dipole moments of atomic "spins" that can be in one of the two states $(+1$ or $-1)$. The model is suitable to use for making a connection between computational hardness of a problem and how difficult it is to solve the corresponding physical system. 

Assume that you have two magnets fixed on the same axis with opposite poles adjacent, they will naturally anti-align. We can regard them as two binary variables $\sigma_i$ and $\sigma_j$, where we assign the value $+1$ to the variable if the north pole is facing up, and $-1$ otherwise. In the natural, optimal, configuration the product of the two binary variables become

$$\sigma_i\sigma_j = -1.$$

Let's regard this as the energy of the considered system, and specifically for this case, this is the lowest energy possible which we refer to as the *ground-state* energy $E_{GS}$. If we were to consider all possible states, *sites*, of the system and sum up their pair-wise energies, we would get the Hamiltonian $H$ of the system, as follows

$$H = \sum_{i,j}\sigma_i\sigma_j.$$

Here we have assumed that one site $\sigma_i$ only interacts with its two nearest neighbors, $\{\sigma_{i-1}, \sigma_{i+1}\}$. But in reality that is not the case. Consider a set $\Gamma$ of sites, for any two adjacents sites $i,j\in\Gamma$ there is an *interaction* $J_{ij}$ and a site $j\in\Gamma$ has an *external magnetic field* $h_j$ interacting with it (which is a sum of the magnetic fields from all other sites). As such, the energy of the configuration of the system is given by the following complete function

$$H = -\sum_{i,j\in\Gamma}J_{ij}\sigma_i\sigma_j - \mu\sum_j h_j\sigma_j$$

where $\mu$ is the magnetic moment (strength and orientation). The negative sign in front of the first summation comes from the fact that all magnetic interactions $J_{ij}$ would be negative if the spins are antiferromagnetic (they behave as one would expect from real magnets).

But what if not all our spins behave like magnets? Then $J_{ij}$ can take both negative and positive values for different pairs. But nature will always strive for the lowest energy configuration.

Let's do some coding, and see what we can learn practically.

## Hamiltonian without external magnetic field $h_j$

In [36]:
def energy_no_external(J, s):
    return -sum(J_ij * s[i] * s[i + 1] for i, J_ij in enumerate(J))


In [37]:
import itertools

J = [1.0, -1.0]

for s in itertools.product(*[{+1, -1} for _ in range(len(J) + 1)]):
    print(f'configuration{s}   \tenergy = {energy_no_external(J, s)}')
    

configuration(1, 1, 1)   	energy = -0.0
configuration(1, 1, -1)   	energy = -2.0
configuration(1, -1, 1)   	energy = -0.0
configuration(1, -1, -1)   	energy = 2.0
configuration(-1, 1, 1)   	energy = 2.0
configuration(-1, 1, -1)   	energy = -0.0
configuration(-1, -1, 1)   	energy = -2.0
configuration(-1, -1, -1)   	energy = -0.0


## Hamiltonian with external magnetic field $h_j$

In [34]:
def energy(J, h, mu, s):
    adjacent_sum = sum(J_ij * s[i] * s[i + 1] for i, J_ij in enumerate(J))
    external_field_sum = sum(h[i] * s[i] for i in h.keys())
    return -adjacent_sum - mu * external_field_sum

def exhaustive_search(J, h, mu):
    for s in itertools.product(*[{+1, -1} for _ in range(len(J) + 1)]):
        print(f'configuration {s}   \tenergy = {energy(J, h, mu, s)}')


In [38]:
J = [1.0, -1.0]
h = {0: 1.5, 1: 2, 2: 1.5}
mu = 1.0

exhaustive_search(J, h, mu)


configuration (1, 1, 1)   	energy = -5.0
configuration (1, 1, -1)   	energy = -4.0
configuration (1, -1, 1)   	energy = -1.0
configuration (1, -1, -1)   	energy = 4.0
configuration (-1, 1, 1)   	energy = 0.0
configuration (-1, 1, -1)   	energy = 1.0
configuration (-1, -1, 1)   	energy = 0.0
configuration (-1, -1, -1)   	energy = 5.0


We can see that depending on the external magnetic field $h_j$ and magnetic moment $\mu$, we get quite different energies of our configurations. 

What we have explored is precisely the full description of the *classical Lenz-Ising model* where the Hamiltonian $H$ describes the energy of a considered system. In combinatorial optimization problems in computer science we are often interested in finding the configuration of a system which yields the lowest cost, risk, or energy. That is precisely what the Ising model can be considered as, however, the problem is often cast as a Quadratic Unconstrained Binary Optimization (QUBO) problem where the only difference is that the binary variables are on the form $\sigma_i\in\{0, 1\}$.

## The transverse-field Ising model

Above we went through the classical Lenz-Ising model, and we can write the same Hamiltonian as above but in quantum mechanical form by defining the *transverse-field Ising model*. Again we consider nearest neighbour interactions for the sites, but they are determined by the alignment or anti-algnment of spin projections along the $z$ axis, with the external magnetic field along the $x$ axis (perpendicular to the $z$ axis). 

Important to note is that the spin projections along the $x$ and $z$ axes are not commuting observable quantities and can thus not be observed simultaneously.

We define the Hamiltonian of such a quantum system by the following formula,

$$H = -J\biggl( \sum_{i,j\in\Gamma}Z_iZ_j + \mu\sum_jX_j\biggr)$$

where $i,j\in\Gamma$ determines the pairs of nearest neigbour sites, $X_i$ and $Z_i$ refer to the Pauli $X$ and $Z$ matrices respectivelly. $J$ is the prefactor with dimensions of energy and $\mu$ is again the coupling coefficient determining the relative strength of the external field.


In [39]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit import execute
from qiskit import Aer
np.set_printoptions(precision=3, suppress=True)


In [45]:
backend = Aer.get_backend('statevector_simulator')
qc = QuantumCircuit(1, 1)


In [47]:
qc.z(0)

job = execute(qc, backend)
state = job.result().get_statevector(qc)

display(state)
display(state.draw('latex'))


Statevector([1.+0.j, 0.+0.j],
            dims=(2,))


<IPython.core.display.Latex object>

In [48]:
qc = QuantumCircuit(1, 1)
qc.x(0)
qc.z(0)

job = execute(qc, backend)
state = job.result().get_statevector(qc)

display(state)
display(state.draw('latex'))


Statevector([ 0.+0.j, -1.-0.j],
            dims=(2,))


<IPython.core.display.Latex object>

In [51]:
def expectation(state, H):
    return float(np.dot(state.T.conj(), np.dot(H, state)).real)


In [53]:
Z = np.array(
    [[1.0,  0.0],
     [0.0, -1.0]],
)
IZ = np.kron(np.eye(2), Z)
ZI = np.kron(Z, np.eye(2))
ZZ = np.kron(Z, Z)
H = -ZZ - 0.5 * (ZI + IZ)
psi = np.kron([[1], [0]], [[1], [0]])
print(f'expectation value = {expectation(psi, H)}')


expectation value = -2.0


We can see that the above Hamiltonian commutes, which in terms mean **all** operators are commutative in the expression. In other words, nothing specifically quantum is going on.

We need to add the term which does not commute, the Pauli $X$ gate. Let's verify this expectation.


In [54]:
qc = QuantumCircuit(1, 1)
qc.x(0)
qc.z(0)

job = execute(qc, backend)
s1 = job.result().get_statevector(qc)


In [56]:
qc = QuantumCircuit(1, 1)
qc.z(0)
qc.x(0)

job = execute(qc, backend)
s2 = job.result().get_statevector(qc)


In [60]:
display(s1)
display(s1.draw('latex'))

display(s2)
display(s2.draw('latex'))

print(f'Pauli-X → Pauli-Z: {s1}')
print(f'Pauli-Z → Pauli-X: {s2}')


Statevector([ 0.+0.j, -1.-0.j],
            dims=(2,))


<IPython.core.display.Latex object>

Statevector([0.+0.j, 1.+0.j],
            dims=(2,))


<IPython.core.display.Latex object>

Pauli-X → Pauli-Z: Statevector([ 0.+0.j, -1.-0.j],
            dims=(2,))
Pauli-Z → Pauli-X: Statevector([0.+0.j, 1.+0.j],
            dims=(2,))


Now we can see a clear difference in outcomes, since the two operators $X_j$ and $Z_j$ do not commute. You should check out the Quantum Approximate Optimization Algorithm (QAOA) notebook for a further deep-dive in Hamiltonians and ground-states of combinatorial systems.
